# T4 - Diseases

Diseases are the cornerstone of almost any Starsim analysis. This is where you add details about what disease you are modeling, including transmissibility, natural history, and mortality outcomes.      

NOTE 1: The version of Starsim was originally conceived as a model for co-circulating STIs, and the current version of the codebase has much more developed models of STIs (including syphilis and HIV) than it does for other pathogens. However, the library of available diseases will keep expanding with time, and we will update these tutorials as it does.

NOTE 2: Starsim descended from a family of similar agent-based models that share common architecture and design principles: [Covasim](https://covasim.org/), [HPVsim](https://hpvsim.org/), and [FPsim](https://fpsim.org/). If you are especially interested in modeling COVID-19, HPV and cervical cancer, or family planning, you may be better off using one of these pre-existing models. For all other diseases/health conditions, Starsim is probably your best bet.


## Overview of Starsim's disease structure

The basic template for modeling a disease in Starsim is the `Disease` class. Much like sims or networks, a `Disease` can be customized by passing in a `pars` dictionary containing parameters. THe `Disease` module dose lots of different things, but three of the model fundamental are:
- `set_initial_states`, which initializes people into states (e.g. susceptible, infected, recovered)
- `make_new_cases`, which makes new cases (e.g., by finding the susceptible contacts of all infected people and calculating the probability of transmission for each)
- `set_prognoses`, which sets the outcomes for people who get infected (e.g., by setting their date of recovery or death).

It will be most helpful to look at an example. Let's look at the SIR model example again: 

In [ ]:
import starsim as ss
network = ss.random()
sir = ss.SIR()
sim = ss.Sim(n_agents=2_000, diseases=sir, networks=network)
sim.run()

We already saw that this model creates results that are stored in `sim.results.sir`. The results can also be directly accessed via `sir.results`. 

For more detail on any of the diseases that are in the Starsim library of diseases, please refer to the docstrings and source code of the disease files. 

## Making your own disease
If you want to make your own disease, you could either inherit from one of the templates in diseases.py, or you could copy the examples and extend them to capture features of the disease that you want to model. For example, here's how you could extend the SIR model to be an SEIR model:

In [ ]:
import numpy as np

class SEIR(ss.SIR):
    def __init__(self, pars=None, *args, **kwargs):
        pars = ss.omerge({
            'dur_exp': 0.5,  # Additional pars beyond the SIR model ones
        }, pars)
        
        # Initialize the SIR model, which will add all the parameters and states of that model.
        super().__init__(pars=pars, *args, **kwargs)

        # Additional states beyond the SIR ones 
        self.exposed = ss.State('exposed', bool, False)
        self.ti_exposed = ss.State('ti_exposed', float, np.nan)

        return

    def update_pre(self, sim):
        # Updates from the SIR model 
        n_deaths = super().update_pre(sim)

        # Additional updates: progress exposed -> infected
        infected = ss.true(self.exposed & (self.ti_infected <= sim.year))
        self.exposed[infected] = False
        self.infected[infected] = True

        return n_deaths

    def update_death(self, sim, uids):
        super().update_death(sim, uids)
        self.exposed[uids] = False
        return

    def infect(self, sim, uids, from_uids):
        super().set_prognoses(sim, uids, from_uids)

        # Carry out state changes associated with infection
        self.susceptible[uids] = False
        self.exposed[uids] = True
        self.ti_exposed[uids] = sim.year

        # Calculate and schedule future outcomes
        dur_exp = self.pars['dur_exp'].rvs(uids)
        self.ti_infected[uids] = sim.year + dur_exp
        dur_inf = self.pars['dur_inf'].rvs(uids)
        will_die = self.pars['p_death'].rvs(uids)        
        self.ti_recovered[uids[~will_die]] = sim.year + dur_inf[~will_die]
        self.ti_dead[uids[will_die]] = sim.year + dur_inf[will_die]

        # Update result count of new infections 
        self.results['new_infections'][sim.ti] += len(uids)
        return
